MAKING OPTICALFLOW FILES

In [7]:
import os
import cv2
import numpy as np

def compute_optical_flow_per_class(input_dir, output_dir):
    """
    Compute optical flow for all frames in the input directory and save in the output directory.
    Args:
        input_dir (str): Path to the input class folder (e.g., Train/Abuse).
        output_dir (str): Path to the output class folder for optical flow.
    """
    os.makedirs(output_dir, exist_ok=True)
    frame_files = sorted([f for f in os.listdir(input_dir) if f.endswith(".png")])
    
    prev_frame = cv2.imread(os.path.join(input_dir, frame_files[0]))
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

    for i in range(1, len(frame_files)):
        curr_frame = cv2.imread(os.path.join(input_dir, frame_files[i]))
        curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)

        # Compute optical flow
        flow = cv2.calcOpticalFlowFarneback(prev_gray, curr_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])

        # Convert to HSV for visualization
        hsv = np.zeros_like(prev_frame)
        hsv[..., 1] = 255
        hsv[..., 0] = ang * 180 / np.pi / 2
        hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
        flow_image = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

        # Save optical flow image
        flow_filename = f"flow_{i:04d}.png"
        cv2.imwrite(os.path.join(output_dir, flow_filename), flow_image)

        prev_gray = curr_gray

def generate_optical_flow_for_dataset(root_dir, output_root_dir, categories):
    """
    Generate optical flow for the entire dataset organized by categories.
    Args:
        root_dir (str): Root directory of the dataset (e.g., Train or Test).
        output_root_dir (str): Root directory to save optical flow frames.
        categories (list): List of class names (subfolder names).
    """
    for category in categories:
        input_dir = os.path.join(root_dir, category)
        output_dir = os.path.join(output_root_dir, category)
        print(f"Processing class: {category}")
        compute_optical_flow_per_class(input_dir, output_dir)
        print(f"Optical flow generated for class: {category}")

# Example usage
categories = [ "RoadAccidents", ]

# Generate optical flow for Train and Test sets
generate_optical_flow_for_dataset("Train", "Train/OpticalFlow", categories)
generate_optical_flow_for_dataset("Test", "Test/OpticalFlow", categories)


Processing class: RoadAccidents
Optical flow generated for class: RoadAccidents
Processing class: RoadAccidents
Optical flow generated for class: RoadAccidents


MAKING AND SAVING NPY FILE

In [ ]:
#without chunking complete files into npy format
import os

import numpy as np
from PIL import Image
from torchvision import transforms

def save_as_numpy(root_dir, output_dir, categories, sequence_length, stride, transform):
    """
    Preprocess frames and save them as .npy files for each class.

    Args:
        root_dir (str): Root directory of the dataset (e.g., Train or Test).
        output_dir (str): Directory to save the .npy files.
        categories (list): List of class names.
        sequence_length (int): Number of frames in each sequence.
        stride (int): Step size for creating overlapping sequences.
        transform (callable): Transformations to apply to each frame.
    """
    os.makedirs(output_dir, exist_ok=True)

    for category in categories:
        category_path = os.path.join(root_dir, category)
        frame_files = sorted([os.path.join(category_path, f) for f in os.listdir(category_path) if f.endswith(".png")])
        print(f"Processing category: {category} ({len(frame_files)} frames)")

        sequences = []
        for i in range(0, len(frame_files) - sequence_length + 1, stride):
            frames = [transform(Image.open(frame).convert("RGB")) for frame in frame_files[i:i + sequence_length]]
            sequences.append(torch.stack(frames, dim=1).numpy())  # Convert to numpy array

        output_file = os.path.join(output_dir, f"{category}.npy")
        np.save(output_file, np.array(sequences))  # Save all sequences for the category
        print(f"Saved {len(sequences)} sequences to {output_file}")

# Define categories
categories = ["Abuse", "Arson", "Burglary", "Fighting", "Robbery", "Shoplifting", "Stealing"]

# Transformation for preprocessing
transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

# Preprocess and save as .npy files
save_as_numpy(root_dir="Train/OpticalFlow", output_dir="Preprocessed/optical/Train", categories=categories, sequence_length=16, stride=8, transform=transform)
save_as_numpy(root_dir="Test/OpticalFlow", output_dir="Preprocessed/optical/Test", categories=categories, sequence_length=16, stride=8, transform=transform)


In [ ]:
#chunking the files into npy format
import os
import numpy as np

def split_npy_into_chunks(input_file, output_dir, chunk_size):
    """
    Split a large .npy file into smaller chunks.
    Args:
        input_file (str): Path to the input .npy file.
        output_dir (str): Directory to save the smaller chunks.
        chunk_size (int): Number of sequences per chunk.
    """
    os.makedirs(output_dir, exist_ok=True)
    data = np.load(input_file)

    for i in range(0, len(data), chunk_size):
        chunk = data[i:i + chunk_size]
        chunk_file = os.path.join(output_dir, f"{os.path.basename(input_file).replace('.npy', '')}_chunk_{i // chunk_size}.npy")
        np.save(chunk_file, chunk)
        print(f"Saved chunk {i // chunk_size} to {chunk_file}")

# Example Usage
categories = ["Abuse", "Arson", "Burglary", "Fighting", "Robbery", "Shoplifting", "Stealing"]
chunk_size = 1000  # Adjust based on your memory limits

# Split train and test datasets
for category in categories:
    split_npy_into_chunks(
        input_file=f"Preprocessed/optical/Train/{category}.npy",
        output_dir=f"PreprocessedChunks/OpticalTrain/{category}",
        chunk_size=chunk_size
    )
    split_npy_into_chunks(
        input_file=f"Preprocessed/optical/Test/{category}.npy",
        output_dir=f"PreprocessedChunks/OpticalTest/{category}",
        chunk_size=chunk_size
    )


BEGINING

In [2]:
import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import os
import numpy as np
from PIL import Image
from torchvision import transforms
import torch.nn as nn
from torchvision.models.video import r2plus1d_18
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import DataLoader
from tqdm import tqdm


PREPROCESSING FOR MODEL

In [3]:
from torch.utils.data import Dataset
import numpy as np
import os

class ChunkedNumpyDataset(Dataset):
    def __init__(self, chunk_dir, categories):
        """
        Args:
            chunk_dir (str): Directory containing chunked .npy files.
            categories (list): List of class names.
        """
        self.chunks = []
        self.labels = []

        # Load all chunk paths and their sequence counts
        for label, category in enumerate(categories):
            category_dir = os.path.join(chunk_dir, category)
            chunk_files = sorted([os.path.join(category_dir, f) for f in os.listdir(category_dir) if f.endswith(".npy")])
            
            for chunk_file in chunk_files:
                sequences = np.load(chunk_file, mmap_mode="r")  # Load sequence metadata without fully loading
                self.chunks.append((chunk_file, len(sequences)))
                self.labels.extend([label] * len(sequences))

        # Precompute cumulative lengths for fast indexing
        self.lengths = [chunk[1] for chunk in self.chunks]
        self.cumulative_lengths = np.cumsum(self.lengths)

    def __len__(self):
        return sum(self.lengths)

    def __getitem__(self, idx):
        # Identify which chunk the index belongs to
        chunk_idx = np.searchsorted(self.cumulative_lengths, idx, side="right")
        relative_idx = idx if chunk_idx == 0 else idx - self.cumulative_lengths[chunk_idx - 1]

        # Load the specific chunk and access the required sequence
        chunk_file, _ = self.chunks[chunk_idx]
        data = np.load(chunk_file, mmap_mode="r")
        sequence = torch.tensor(data[relative_idx])
        label = self.labels[idx]
        return sequence, label


In [4]:
class AlignedMultiStreamDataset(Dataset):
    def __init__(self, rgb_dataset, flow_dataset):
        """
        Combines RGB and Optical Flow datasets into a single aligned dataset.
        Args:
            rgb_dataset: Dataset object for RGB frames.
            flow_dataset: Dataset object for Optical Flow frames.
        """
        self.rgb_dataset = rgb_dataset
        self.flow_dataset = flow_dataset
        self.length = min(len(rgb_dataset), len(flow_dataset))  # Match lengths

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        # Retrieve RGB and flow data
        rgb_data, rgb_label = self.rgb_dataset[idx]
        flow_data, flow_label = self.flow_dataset[idx]

        # Ensure labels match
        assert rgb_label == flow_label, f"Label mismatch: RGB({rgb_label}) vs Flow({flow_label})"

        # Return separate tensors for RGB and Flow
        return (rgb_data, flow_data), rgb_label


DATALOADER

In [5]:
import time
from torch.utils.data import DataLoader

categories = ["Abuse", "Arson", "Burglary", "Fighting", "Robbery", "Shoplifting", "Stealing"]

# Dataset directories
rgb_train_dir = "PreprocessedChunks/Train/"
flow_train_dir = "PreprocessedChunks/OpticalTrain/"
rgb_test_dir = "PreprocessedChunks/Test/"
flow_test_dir = "PreprocessedChunks/OpticalTest/"


# Initialize datasets
train_rgb_dataset = ChunkedNumpyDataset(chunk_dir=rgb_train_dir, categories=categories)
train_flow_dataset = ChunkedNumpyDataset(chunk_dir=flow_train_dir, categories=categories)
train_dataset = AlignedMultiStreamDataset(train_rgb_dataset, train_flow_dataset)


test_rgb_dataset = ChunkedNumpyDataset(chunk_dir=rgb_test_dir, categories=categories)
test_flow_dataset = ChunkedNumpyDataset(chunk_dir=flow_test_dir, categories=categories)
test_dataset = AlignedMultiStreamDataset(test_rgb_dataset, test_flow_dataset)

# DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)


MODEL

In [5]:


class MultiStreamR2Plus1D(nn.Module):
    def __init__(self, num_classes):
        super(MultiStreamR2Plus1D, self).__init__()
        
        # RGB Stream
        self.rgb_stream = r2plus1d_18(pretrained=True)
        self.rgb_stream.fc = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(self.rgb_stream.fc.in_features, num_classes)
        )
        
        # Optical Flow Stream
        self.flow_stream = r2plus1d_18(pretrained=True)
        self.flow_stream.fc = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(self.flow_stream.fc.in_features, num_classes)
        )
        
        # Fusion Layer
        self.fc_fusion = nn.Linear(2 * num_classes, num_classes)


    def forward(self, rgb_input, flow_input):
        # Forward pass through RGB and flow streams
        rgb_out = self.rgb_stream(rgb_input)
        flow_out = self.flow_stream(flow_input)

        # Concatenate outputs and pass through fusion layer
        combined_out = torch.cat((rgb_out, flow_out), dim=1)
        return self.fc_fusion(combined_out)


TRAINING

In [9]:
# Model initialization
num_classes = len(categories)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MultiStreamR2Plus1D(num_classes=num_classes).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
scaler = GradScaler(init_scale=65536.0)


# Function to save model checkpoint
def save_checkpoint(epoch, model, optimizer, scheduler, scaler, save_dir="checkpoints"):
    os.makedirs(save_dir, exist_ok=True)
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'scaler_state_dict': scaler.state_dict() if scaler is not None else None,
    }
    save_path = os.path.join(save_dir, f"checkpoint_epoch_{epoch}.pth")
    torch.save(checkpoint, save_path)
    print(f"Checkpoint saved at {save_path}")

# Training loop
epochs = 1
save_dir = "checkpoints"
for epoch in range(1, epochs + 1):
    model.train()
    running_loss = 0.0

    # Progress bar for training
    progress_bar = tqdm(train_loader, desc=f"Training Epoch {epoch}/{epochs}")
    for (rgb_data, flow_data), labels in progress_bar:
        rgb_data, flow_data, labels = rgb_data.to(device), flow_data.to(device), labels.to(device)

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass with mixed precision
        with autocast():
            outputs = model(rgb_data, flow_data)
            loss = criterion(outputs, labels)

        # Backward pass
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    # Step the scheduler
    scheduler.step()

    # Print epoch loss
    print(f"Epoch [{epoch}/{epochs}] Loss: {running_loss / len(train_loader):.4f}")

    # Save model checkpoint
    save_checkpoint(epoch, model, optimizer, scheduler, scaler, save_dir)

# Evaluation loop
model.eval()
correct = 0
total = 0
running_loss = 0.0

# Progress bar for evaluation
with torch.no_grad():
    progress_bar = tqdm(test_loader, desc="Evaluating")
    for (rgb_data, flow_data), labels in progress_bar:
        rgb_data, flow_data, labels = rgb_data.to(device), flow_data.to(device), labels.to(device)

        # Forward pass
        with autocast():
            outputs = model(rgb_data, flow_data)
            loss = criterion(outputs, labels)

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        running_loss += loss.item()

# Print ev


C:\Users\adars\AppData\Local\Temp\ipykernel_24324\3471267753.py:10: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(init_scale=65536.0)
Training Epoch 1/1:   0%|          | 0/1709 [00:00<?, ?it/s]C:\Users\adars\AppData\Local\Temp\ipykernel_24324\3471267753.py:43: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Training Epoch 1/1:   2%|▏         | 36/1709 [23:46<18:24:57, 39.63s/it, loss=1.12]


KeyboardInterrupt: 

LOADING MODEL

In [ ]:
def load_checkpoint(checkpoint_path, model, optimizer, scheduler, scaler):
    """
    Load a model checkpoint to resume training.
    
    Args:
        checkpoint_path (str): Path to the saved checkpoint file.
        model (torch.nn.Module): The model to load the weights into.
        optimizer (torch.optim.Optimizer): The optimizer to load the state into.
        scheduler (torch.optim.lr_scheduler): The scheduler to load the state into.
        scaler (torch.cuda.amp.GradScaler): The gradient scaler to load the state into.
    
    Returns:
        int: The epoch to resume training from.
    """
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    if scaler is not None and checkpoint['scaler_state_dict'] is not None:
        scaler.load_state_dict(checkpoint['scaler_state_dict'])
    print(f"Checkpoint loaded: {checkpoint_path}")
    return checkpoint['epoch']


In [1]:
import torch
torch.cuda.empty_cache()
